In [12]:
import numpy as np
import arviz as az
import matplotlib.pyplot as plt
from likelihood_mesonh import likelihood_mesonh
import sys  # to put the SCM into the PYTHONPATH
sys.path.append('edmf_ocean/library/F2PY')
from sys import exit
import time as TIME
import xarray as xr
from scipy.interpolate import interp1d
import scipy.signal
from scm_class import SCM
from netCDF4 import Dataset
import matplotlib.pyplot as plt
import numpy as np
from case_configs import case_params, default_params
from multiprocess import Pool #multiprocessING cannot handle locally defined functions, multiprocess can
import subprocess
from test_version_edmf_ocean import check_edmf_ocean_version
check_edmf_ocean_version()
from interpolate_LES_on_SCM_grids import regrid_and_save


## Load the data
data = az.from_netcdf('MCMC_output/MAP_error_run.nc')

## convert to xarray dataset
ds = az.convert_to_dataset(data)

In [2]:
ds['likelihood'][0].max()
ds['likelihood'][0].min()

<xarray.DataArray 'likelihood' ()>
array(-2.9629237e+14)
Coordinates:
    chain    int64 0

In [5]:
ds

<xarray.Dataset>
Dimensions:     (chain: 4, draw: 40000)
Coordinates:
  * chain       (chain) int64 0 1 2 3
  * draw        (draw) int64 0 1 2 3 4 5 ... 39994 39995 39996 39997 39998 39999
Data variables:
    Cdet        (chain, draw) float64 ...
    Cent        (chain, draw) float64 ...
    bc_ap       (chain, draw) float64 ...
    delta_bkg   (chain, draw) float64 ...
    likelihood  (chain, draw) float64 ...
    log_wp_0    (chain, draw) float64 ...
    up_c        (chain, draw) float64 ...
    wp_0        (chain, draw) float64 ...
    wp_a        (chain, draw) float64 ...
    wp_b        (chain, draw) float64 ...
    wp_bp       (chain, draw) float64 ...
Attributes:
    created_at:                 2024-10-26T13:31:28.078112+00:00
    arviz_version:              0.19.0
    inference_library:          pymc
    inference_library_version:  5.16.2
    sampling_time:              57011.942348480225
    tuning_steps:               3000

In [21]:
slicer = ds['likelihood'][0] > 0.1 * ds['likelihood'][0].max() 
ds['likelihood'][0][slicer]
ds_sliced = { key: ds[key][0, slicer] for key in ds.data_vars.keys() }

nsubsample = len(ds_sliced['likelihood'])
number_of_draws = 10
subsample_indices = np.random.choice(nsubsample, number_of_draws, replace=False)
ds_subsampled = {key: ds_sliced[key][subsample_indices] for key in ds_sliced.keys()}

# ds_sliced['likelihood'][subsample_indices]
ds_subsampled



{'Cdet': <xarray.DataArray 'Cdet' (draw: 10)>
 [10 values with dtype=float64]
 Coordinates:
     chain    int64 0
   * draw     (draw) int64 38693 37486 37868 37800 ... 37199 38219 39402 38135,
 'Cent': <xarray.DataArray 'Cent' (draw: 10)>
 [10 values with dtype=float64]
 Coordinates:
     chain    int64 0
   * draw     (draw) int64 38693 37486 37868 37800 ... 37199 38219 39402 38135,
 'bc_ap': <xarray.DataArray 'bc_ap' (draw: 10)>
 [10 values with dtype=float64]
 Coordinates:
     chain    int64 0
   * draw     (draw) int64 38693 37486 37868 37800 ... 37199 38219 39402 38135,
 'delta_bkg': <xarray.DataArray 'delta_bkg' (draw: 10)>
 [10 values with dtype=float64]
 Coordinates:
     chain    int64 0
   * draw     (draw) int64 38693 37486 37868 37800 ... 37199 38219 39402 38135,
 'likelihood': <xarray.DataArray 'likelihood' (draw: 10)>
 [10 values with dtype=float64]
 Coordinates:
     chain    int64 0
   * draw     (draw) int64 38693 37486 37868 37800 ... 37199 38219 39402 38135,
 'log_

In [35]:
def run_scm(params_to_estimate,case,draw):
    # ====================================Run the SCM cases=======================================
    params = default_params.copy()  # Create a copy of default_params
    params.update(case_params[case])  # Update with the specific case hyperparameters in case_params[case]
    params.update(params_to_estimate) # Update with the parameters to estimate 
    params.update( {'output_filename': 'plotting_outputs/'+case+str(draw)+'.nc','write_netcdf': True})
    print('Running SCM with the following parameters:', params)
    scm = SCM(**params)
    scm.run_direct()            # Run the SCM

In [23]:
ds_subsampled.remove('likelihood')

AttributeError: 'dict' object has no attribute 'remove'

In [36]:
cases = ['FC500','W005_C500_NO_COR']

for draw in range(number_of_draws):
    for case in cases:
        params_to_estimate = {key: ds_subsampled[key][draw] for key in ds_subsampled.keys() if (key != 'likelihood' and key != 'log_likelihood' and key != 'log_wp_0')}
        params_to_estimate['wp0'] = params_to_estimate.pop('wp_0')  # Rename 'old_key' to 'new_key'

        run_scm(params_to_estimate=params_to_estimate,case=case,draw=draw)

In [32]:
params_to_estimate = {key: ds_subsampled[key][draw] for key in ds_subsampled.keys() if (key != 'likelihood' and key != 'log_likelihood' and key != 'log_wp_0')}

In [31]:
params_to_estimate.keys()

dict_keys(['Cdet', 'Cent', 'bc_ap', 'delta_bkg', 'log_wp_0', 'up_c', 'wp_0', 'wp_a', 'wp_b', 'wp_bp'])

In [19]:
ds_subsampled.keys()

dict_keys(['Cdet', 'Cent', 'bc_ap', 'delta_bkg', 'likelihood', 'log_wp_0', 'up_c', 'wp_0', 'wp_a', 'wp_b', 'wp_bp'])